<a href="https://colab.research.google.com/github/inseungkang/learningalgos/blob/main/readc3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>